In [1]:
# 필요 패키지 로드
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches
%matplotlib inline
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols 
from statsmodels.api import OLS, add_constant

In [3]:
Basic_dir = os.path.dirname(os.path.dirname(os.path.realpath('EDA.ipynb')))
PATH = Basic_dir + '\\Data\\Data(WineQ_B)'
os.listdir(PATH)

['sample_submission.csv', 'test.csv', 'train.csv']

In [56]:
# 데이터 로드
train = pd.read_csv(PATH + '\\train.csv')
test = pd.read_csv(PATH + '\\test.csv')
print(train.columns)
train.head()

Index(['id', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'type',
       'quality'],
      dtype='object')


,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,1,8.0,0.18,0.37,1.3,0.040,15.0,96.0,0.99120,3.06,0.61,12.1,white,6
1,2,7.5,0.38,0.29,4.9,0.021,38.0,113.0,0.99026,3.08,0.48,13.0,white,7
2,3,6.1,0.27,0.33,2.2,0.021,26.0,117.0,0.98860,3.12,0.30,12.5,white,6
3,4,6.4,0.32,0.50,10.7,0.047,57.0,206.0,0.99680,3.08,0.60,9.4,white,5
4,5,8.4,0.37,0.43,2.3,0.063,12.0,19.0,0.99550,3.17,0.81,11.2,red,7


In [10]:
df = train.drop(['id'], axis = 1).copy()
# 범주형 변수 인코딩 : 와인 종류 - 0 : 화이트, 1: 레드
df.type = np.where(df.type == 'white', 0, 1)
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'type', 'quality'],
      dtype='object')

In [11]:
x_list = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
          'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
          'pH', 'sulphates', 'alcohol']
df

In [41]:
# 순위변수(품질)와 연속형변수(x_list에 해당하는 자료들)을 분석하기 위해 스피어만 상관계수 사용
# 품질을 연속형으로 보면 회귀 및 피어슨 상관계수분석 가능
# p-value가 0.05 이하인 변수들은 제외
from scipy import stats
val_list = []
for col in x_list:
    if stats.spearmanr(df[col], df.quality)[1] < 0.05:
        val_list.append(col)
        print(f'{col} : {round(stats.spearmanr(df[col], df.quality)[0], 3)}, {round(stats.spearmanr(df[col], df.quality)[1], 5)}')
val_list = val_list + ['type']

fixed acidity : -0.109, 0.0
volatile acidity : -0.254, 0.0
citric acid : 0.08, 1e-05
chlorides : -0.286, 0.0
free sulfur dioxide : 0.082, 0.0
total sulfur dioxide : -0.058, 0.00092
density : -0.329, 0.0
pH : 0.044, 0.01194
alcohol : 0.449, 0.0


In [42]:
df_train01 = df[val_list + ['quality']].copy()
df_train01.head()

,fixed acidity,volatile acidity,citric acid,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,alcohol,type,quality
0,8.0,0.18,0.37,0.040,15.0,96.0,0.99120,3.06,12.1,0,6
1,7.5,0.38,0.29,0.021,38.0,113.0,0.99026,3.08,13.0,0,7
2,6.1,0.27,0.33,0.021,26.0,117.0,0.98860,3.12,12.5,0,6
3,6.4,0.32,0.50,0.047,57.0,206.0,0.99680,3.08,9.4,0,5
4,8.4,0.37,0.43,0.063,12.0,19.0,0.99550,3.17,11.2,1,7


In [43]:
# 로지스틱 회귀분석 테스트
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# 클래스 불균형 처리를 해주는 imbalanced-learn를 호출
# 오버 샘플링 시행
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
ros = RandomOverSampler()
ov_data, ov_label = ros.fit_resample(df_train01[val_list].copy(), df_train01[['quality']].copy())
ov_data = pd.DataFrame(ov_data, columns = val_list)
ov_data['quality'] = ov_label
len(df_train01), len(ov_data)

(3231, 7090)

In [44]:
ov_data.quality.value_counts()

6    1418
7    1418
5    1418
8    1418
4    1418
Name: quality, dtype: int64

In [49]:
# 모델 로드 / 데이터 분할
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
X_train, X_test, y_train, y_test = train_test_split(ov_data[val_list], ov_data[['quality']], test_size = 0.2, stratify = ov_data.quality, random_state = 7)

In [52]:
lr.fit(X_train, y_train)

C:\Users\Bestc\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Bestc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [55]:
# 성능평가
print(confusion_matrix(y_test, lr.predict(X_test)))
print(classification_report(y_test, lr.predict(X_test)))

[[157  63  24  25  14]
 [ 98 100  44  15  27]
 [ 69  57  48  41  69]
 [ 54  35  32  69  93]
 [ 38  32  38  50 126]]
              precision    recall  f1-score   support

           4       0.38      0.55      0.45       283
           5       0.35      0.35      0.35       284
           6       0.26      0.17      0.20       284
           7       0.34      0.24      0.29       283
           8       0.38      0.44      0.41       284

    accuracy                           0.35      1418
   macro avg       0.34      0.35      0.34      1418
weighted avg       0.34      0.35      0.34      1418



In [ ]:
# 성능평가(테스트데이터 셋)
print(confusion_matrix(y_test, lr.predict(X_test)))
print(classification_report(y_test, lr.predict(X_test)))